In [1]:
import open3d
import numpy as np
import pandas as pd
import os
import glob
import tqdm
import copy

import utils.registration as registration
import utils.fread as fread
import utils.transform as transform
import utils.grid_search_unopt as grid_search
import utils.functions as functions

from time import sleep
from PIL import Image
from utils.config import Config

In [7]:
config = Config(
    sequence_dir="../local-registration/data/raw_data",
    feature_dir="../local-registration/data/features",
    output_dir="data/trajectories/local/FPFH",
    experiment="exp_11",
    trial="trial_1",
    subject="subject-1",
    sequence="01",
    groundtruth_dir="data/trajectories/groundtruth",
)

config.voxel_size=0.03
config.target_fps=20
config.min_std=0.5

In [8]:
sequence_dir = config.get_sequence_dir()
feature_dir = config.get_feature_dir()

sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)
num_frames = len(sequence_ts)

In [9]:
std_values = []

for t in tqdm.trange(num_frames):
    depth_img_file = os.path.join(sequence_dir, f"frame-{sequence_ts[t]}.depth.png")
    std_values.append(registration.calc_std(depth_img_file, 4000))
    
std_values = np.array(std_values)

device_0_ts = fread.get_timstamps_from_images(os.path.join(config.get_global_dir(), "device-0"), ext=".depth.png")
device_1_ts = fread.get_timstamps_from_images(os.path.join(config.get_global_dir(), "device-1"), ext=".depth.png")
device_2_ts = fread.get_timstamps_from_images(os.path.join(config.get_global_dir(), "device-2"), ext=".depth.png")

y = [[], [], []]

for i in range(num_frames):
    y[0].append(functions.nearest(device_0_ts, sequence_ts[i]))
    y[1].append(functions.nearest(device_1_ts, sequence_ts[i]))
    y[2].append(functions.nearest(device_2_ts, sequence_ts[i]))
    
    y[0][i] = np.abs(y[0][i] - sequence_ts[i])
    y[1][i] = np.abs(y[1][i] - sequence_ts[i])
    y[2][i] = np.abs(y[2][i] - sequence_ts[i])
    
global_frame_delays = np.array(y)
global_frame_delays_inds = np.ones(global_frame_delays.shape, dtype=np.int8)

for r, c in np.argwhere(global_frame_delays > 100):
    global_frame_delays_inds[r, c] = 0
    
global_frame_delays_inds = np.sum(global_frame_delays_inds, axis=0)
global_frame_delays_inds = np.where(global_frame_delays_inds == 3, 1, 0)
    
global_pos = [0]
for t in tqdm.trange(num_frames):

    if t - global_pos[-1] >= config.target_fps * 0.8: 
        global_pos.append(t)
        continue
    
    if (np.abs(std_values[t] - std_values[global_pos[-1]]) > config.delta) and (t - global_pos[-1] > config.target_fps * 0.5):
        global_pos.append(t)

global_pos = np.array(global_pos)

100%|██████████| 420/420 [00:00<00:00, 420732.67it/s]


In [10]:
t = 20
source_feature_file = os.path.join(feature_dir, f"{sequence_ts[global_pos[t]]}.secondary.npz")
target_feature_file = os.path.join(feature_dir, f"{sequence_ts[global_pos[t]]}.global.npz")

source, target, reg_result = grid_search.global_registration(source_feature_file, target_feature_file, config.voxel_size, cell_size=2, refine_enabled=True)

if reg_result:
    registration.describe(source, target, reg_result)
else:
    print("Registration failed.")

Keypts: [19649, 82704]	No of matches: 10418	Fitness: 0.5302	Inlier RMSE: 0.0218


In [11]:
registration.view(source, target, reg_result.transformation if reg_result else np.identity(4))